In [1]:
import pandas as pd
import numpy as np
import torch
import os
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc = nn.Linear(n_features, 1)

    def forward(self, x):
        return self.fc(x)

df = pd.read_csv('./dataset/room_info_reform.csv')

df = df[df['target']!='uninspected']
display(df)
# データフレームを訓練データとテストデータに分割
X = df.drop(['room_num', 'target'], axis=1)
y = df['target'].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# いくつかのモデルを試す
models = [
    {"name": "RF", "model": RandomForestRegressor()}
]

for m in models:
    model = m["model"]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Model: {m['name']}")
    print(f"Mean Squared Error: {mse}")

    print("\n-----------------------------------\n")

# PyTorch part starts here
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
model_path = './learned_model/torch_model.pth'
model = Net(X_train.shape[1])  # 新たなモデルを作成

# .pthファイルが存在するか確認
if os.path.isfile(model_path):
    # ファイルが存在する場合はモデルの状態を読み込む
    model.load_state_dict(torch.load(model_path))
    print("Model loaded from {}".format(model_path))

model.eval()  # モデルを評価モードに設定（dropout等を無効化）
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

model.eval()
with torch.no_grad():
    outputs = model(X_test)
    mse = criterion(outputs, y_test).item()
    print(f"Model: PyTorch NN")
    print(f"Mean Squared Error: {mse}")
    print("\n-----------------------------------\n")

torch.save(model.state_dict(), model_path)


,room_num,room_v,room_h,target,bed_1_exist,bed_1_v_width,bed_1_h_width,bed_1_x,bed_1_y,bed_1_rotation,...,chest_3_d_plant_3,chest_3_d_shelf_1,chest_3_d_shelf_2,chest_3_d_shelf_3,chest_3_d_chest_1,chest_3_d_chest_2,chest_3_d_chest_3,chest_3_d_bed_1,chest_3_d_bed_2,chest_3_d_bed_3
1,room_2,5,5,10,1,0.5,1.4,5.99,3.00,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,room_3,4,6,10,1,0.5,1.4,6.99,3.00,90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,room_4,5,5,10,1,0.5,1.4,1.01,4.00,270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,room_5,3,6,15,1,0.5,1.4,6.99,3.00,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,room_6,6,3,8,1,0.5,1.4,1.01,5.00,360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,room_136,3,6,0,1,0.5,1.4,3.00,1.01,360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,room_137,3,6,15,1,0.5,1.4,5.00,3.99,270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,room_138,5,3,0,1,0.5,1.4,3.99,2.00,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,room_139,3,4,10,0,0.0,0.0,0.00,0.00,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Model: RF
Mean Squared Error: 69.26425714285715

-----------------------------------

Model loaded from ./learned_model/torch_model.pth
Model: PyTorch NN
Mean Squared Error: 84.37288665771484

-----------------------------------

